In [1]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import random
from collections import Counter
from tqdm import tqdm_notebook as tqdm 
# from deap import base, creator, tools, algorithms
import pickle
import multiprocessing
import math
import warnings
import itertools
warnings.filterwarnings("ignore")
from IPython.display import clear_output
import optuna 
import os
import csv
from optuna.storages import RDBStorage 
import copy

In [3]:
##### Importing Distance matrix and linkages
dist_mtx = pd.read_excel('/kaggle/input/data-to-play/VHO-Data File- WO-22-23 (2) (3).xlsx' , sheet_name = 'Real_values_dist_matrix')
link_df = pd.read_excel('/kaggle/input/data-to-play/VHO-Data File- WO-22-23 (2) (3).xlsx' , sheet_name = 'Structured Linkages')
link_df_melt = link_df.melt(id_vars = ['Destination' , 'Product'] , var_name = 'Pickup'  )
link_df_melt = link_df_melt[['Pickup' , 'Destination' , 'Product' , 'value']]

In [4]:
link_df_melt['Pickup'] = link_df_melt['Pickup'].apply(lambda x: x.strip())
link_df_melt['Destination'] = link_df_melt['Destination'].apply(lambda x: x.strip())

In [5]:
link_df_melt['Pickup'].unique() ,

(array(['Chennai', 'Haldia', 'Jamnagar', 'Kandla', 'Mangalore', 'Paradip'],
       dtype=object),)

In [6]:
dist_mtx = pd.read_excel('/kaggle/input/data-to-play/VHO-Data File- WO-22-23 (2) (3).xlsx' , sheet_name = 'Real_values_dist_matrix')
dist_mtx_mlt = dist_mtx.melt(id_vars = 'from_port')
dist_mtx_mlt['from_port'] = dist_mtx_mlt['from_port'].apply(lambda x: x.strip().lower())
dist_mtx_mlt['variable'] = dist_mtx_mlt['variable'].apply(lambda x: x.strip().lower())

link_df = pd.read_excel('/kaggle/input/data-to-play/VHO-Data File- WO-22-23 (2) (3).xlsx' , sheet_name = 'Structured Linkages')
link_df_melt = link_df.melt(id_vars = ['Destination' , 'Product'] , var_name = 'Pickup'  )
link_df_melt = link_df_melt[['Pickup' , 'Destination' , 'Product' , 'value']]
link_df_melt['Pickup'] = link_df_melt['Pickup'].apply(lambda x: x.strip())
link_df_melt['Destination'] = link_df_melt['Destination'].apply(lambda x: x.strip())

coast_map = pd.read_excel("/kaggle/input/data-to-play/port_coast_map (2).xlsx")

In [7]:
 ###### Coast Split Method

link_df_melt = link_df_melt.merge(coast_map , left_on = 'Pickup' , right_on ='port' , how = 'inner')
link_df_melt.rename(columns = {'Coast':'p_cost'} , inplace = True)
link_df_melt = link_df_melt.merge(coast_map , left_on = 'Destination' , right_on ='port' , how = 'inner')
link_df_melt.rename(columns = {'Coast':'d_cost'} , inplace = True)
link_df_melt.drop(columns = ['port_x' , 'port_y'] , inplace = True)
link_df_melt['comb'] = link_df_melt[['p_cost' , 'd_cost']].apply(lambda x: x['p_cost'] + '_' + x['d_cost'] , axis = 1)

In [8]:
link_df_melt_all = link_df_melt.copy(deep = True)

In [9]:
######## Selecting Product for solving 

# link_df_melt = link_df_melt_all[link_df_melt_all['Product'] == 'HSD']

In [10]:
pk_df = link_df_melt.pivot_table(index = ['Pickup'] , columns = 'Product' , aggfunc = 'sum' , values = 'value').reset_index()
pk_df.rename(columns = {'Pickup' : 'port'} , inplace = True)
pk_df = pk_df[[i for i in link_df_melt['Product'].unique()] + ['port']]


dk_df = link_df_melt.pivot_table(index = ['Destination'] , columns = 'Product' , aggfunc = 'sum' , values = 'value').reset_index()
dk_df.rename(columns = {'Destination' : 'port'} , inplace = True)
dk_df = dk_df[[i for i in link_df_melt['Product'].unique()] + ['port']]
for i in  link_df_melt['Product'].unique():
    # dk_df[i] = dk_df[i] * -1
    dk_df[i] = dk_df[i] * 1

pk_df = pk_df.fillna(0)
dk_df = dk_df.fillna(0)

pk_df['type'] = 'pickup'
dk_df['type'] = 'drop'
pd_df = pd.concat([pk_df , dk_df] ).reset_index(drop = True)
pd_df['port'] = pd_df['port'].apply(lambda x:  x.strip().lower())

port_name_dict = dict(zip(dist_mtx_mlt['from_port'].unique().tolist()  , [i + 1 for i in range(len(dist_mtx_mlt['from_port'].unique()))]))


pd_df['comb'] = pd_df[['port' , 'type']].apply(lambda x: (x['port'] ,x['type']) , axis = 1)
all_comb = set([(i[0] , i[1]) for i in  list(itertools.product(pd_df['port'].unique() , pd_df['type'].unique()))])
avl_comb = set(pd_df['comb'].unique())

for i in all_comb.difference(avl_comb):
    # pd_df.loc[len(pd_df)] = [ 0 , i[0] , i[1] , i]
    pd_df.loc[len(pd_df)] = [0 , 0 , 0 , 0 , i[0] , i[1] , i]


pd_df['port_no'] = pd_df['port'].apply(lambda x : port_name_dict[x])



# ########## Single product adjustment
# for prod in ['MS' , 'HFHSD' , 'SKO']:
#     pd_df[prod] = pd_df[prod] * 0
# # pd_df['prod_sum'] = pd_df['MS'] + pd_df['HFHSD']+pd_df['HSD']+pd_df['SKO']
# # pd_df = pd_df[pd_df['prod_sum'] != 0]




# link_dict = {}
# for i in pd_df['port'].unique():
#     sub_pd_df = pd_df[pd_df['port'] == i].copy(deep = True).sort_values(by = 'type' , ascending = False)
#     # sub_pd_df = 
    
#     link_dict[i] = ((sub_pd_df['HFHSD'].values.tolist()[0] , sub_pd_df['HFHSD'].values.tolist()[1])
#                     ,( sub_pd_df['HSD'].values.tolist()[0] ,  sub_pd_df['HSD'].values.tolist()[1]) 
#                     , (sub_pd_df['MS'].values.tolist()[0] , sub_pd_df['MS'].values.tolist()[1])
#                     , (sub_pd_df['SKO'].values.tolist()[0] , sub_pd_df['SKO'].values.tolist()[1]))

In [11]:


# # Remove keys where the sum of all quantities is 0
# link_dict = {k: v for k, v in link_dict.items() if sum(sum(t) for t in v) != 0}

# # Print the cleaned dictionary
# print(link_dict)

In [12]:
########## Structuring distance matrix 

dist_mtx.columns = [i.lower().strip() for i in dist_mtx.columns]
dist_mtx['from_port'] = dist_mtx['from_port'].apply(lambda x: x.lower().strip())
dist_mtx = dist_mtx.set_index(dist_mtx.columns[0])



In [13]:
pd_df

Product,MS,HFHSD,HSD,SKO,port,type,comb,port_no
0,0.000000,0.000000,37.700223,0.000000,chennai,pickup,"(chennai, pickup)",10
1,19.460816,11.575040,0.000000,8.394744,haldia,pickup,"(haldia, pickup)",13
2,0.110670,0.000000,0.000000,0.000000,jamnagar,pickup,"(jamnagar, pickup)",1
3,0.000000,0.000000,41.940287,0.000000,kandla,pickup,"(kandla, pickup)",3
4,21.024172,0.000000,38.244156,0.000000,mangalore,pickup,"(mangalore, pickup)",7
5,15.368213,33.520247,35.831344,0.000000,paradip,pickup,"(paradip, pickup)",12
6,7.296305,3.142570,0.000000,0.000000,chennai,drop,"(chennai, drop)",10
7,18.229813,0.000000,37.700223,0.000000,ennore,drop,"(ennore, drop)",18
8,2.328996,0.000000,12.687256,0.000000,jnpt,drop,"(jnpt, drop)",5
9,0.000000,2.618807,0.000000,0.000000,kakinada,drop,"(kakinada, drop)",16


In [14]:
prodt = [ 'MS' , 'HSD' , 'HFHSD'  , 'SKO']
pd_df_copy = pd_df.copy(deep = True)

pd_df_copy = pd_df_copy[prodt + [ 'port' , 'type']] 

for i in prodt:
    pd_df_copy[i] = np.where(pd_df_copy['type'] == 'drop' , pd_df_copy[i] * -1 , pd_df_copy[i])

pd_df_copy_net_all = pd.pivot_table(pd_df_copy , index = ['port'] , values = prodt , aggfunc = 'sum').reset_index()

# for i in prodt:
#     pd_df_copy_net_all[i] = pd_df_copy_net_all[i].apply(lambda x: round(x))


In [15]:
pd_df_copy_net_all

Product,port,HFHSD,HSD,MS,SKO
0,chennai,-3.142570,37.700223,-7.296305,0.000000
1,ennore,0.000000,-37.700223,-18.229813,0.000000
2,goa,-2.095043,-11.594080,-3.706210,-0.041220
3,haldia,11.575040,0.000000,19.460816,8.394744
4,jamnagar,0.000000,0.000000,0.110670,0.000000
5,jnpt,0.000000,-12.687256,-2.328996,0.000000
6,kakinada,-2.618807,0.000000,0.000000,0.000000
7,kandla,-0.838030,41.940287,-0.110670,0.000000
8,karwar,-5.237520,0.000000,0.000000,0.000000
9,kochi,-6.285140,-26.177182,-4.190838,0.000000


In [16]:
# pd_df_copy_net_all.loc[4,'MS'] = 1
# pd_df_copy_net_all.loc[7,'MS'] = -1
# pd_df_copy_net_all.loc[3,'MS'] = 20

# pd_df_copy_net_all.loc[2,'SKO'] = -1
# pd_df_copy_net_all.loc[3,'SKO'] = 9

# pd_df_copy_net_all.loc[7,'HSD'] = 43


In [17]:
pd_df_copy_net_all.sum()

Product
port     chennaiennoregoahaldiajamnagarjnptkakinadakand...
HFHSD                                                  0.0
HSD                                                    0.0
MS                                                     0.0
SKO                                                    0.0
dtype: object

In [18]:
# prodt = 'MS'
# pd_df_copy = pd_df.copy(deep = True)

# pd_df_copy = pd_df_copy[[prodt , 'port' , 'type']] 

# pd_df_copy[prodt] = np.where(pd_df_copy['type'] == 'drop' , pd_df_copy[prodt] * -1 , pd_df_copy[prodt])

# pd_df_copy_net_ms = pd.pivot_table(pd_df_copy , index = ['port'] , values = [prodt] , aggfunc = 'sum').reset_index()

# # pd_df_copy_net = pd_df_copy_net[pd_df_copy_net['HSD'] != 0]

# pd_df_copy_net_ms 

In [19]:
# prodt = 'HSD'
# pd_df_copy = pd_df.copy(deep = True)

# pd_df_copy = pd_df_copy[[prodt , 'port' , 'type']] 

# pd_df_copy[prodt] = np.where(pd_df_copy['type'] == 'drop' , pd_df_copy[prodt] * -1 , pd_df_copy[prodt])

# pd_df_copy_net_hsd = pd.pivot_table(pd_df_copy , index = ['port'] , values = [prodt] , aggfunc = 'sum').reset_index()

# # pd_df_copy_net = pd_df_copy_net[pd_df_copy_net['HSD'] != 0]

# pd_df_copy_net_hsd

In [20]:
######## Subsetting Distance matrix to remove ports with no linkage

sub_dist_mtx = dist_mtx.reset_index().melt(id_vars = 'from_port') 

sub_dist_mtx['comb'] = sub_dist_mtx[['from_port' , 'variable']].apply(lambda x: (x['from_port'] , x['variable']) , axis = 1)

#### required posr combinations 
req_distances = list(itertools.product(pd_df_copy_net_all['port'].unique() , pd_df_copy_net_all['port'].unique()))

sub_dist_mtx = sub_dist_mtx[sub_dist_mtx['comb'].isin(req_distances)]

sub_dist_mtx.drop(columns = 'comb'  , inplace = True)

sub_dist_mtx = sub_dist_mtx.pivot(index = ['from_port'] , columns = ['variable'] , values = 'value')



In [21]:
sub_dist_mtx

variable,chennai,ennore,goa,haldia,jamnagar,jnpt,kakinada,kandla,karwar,kochi,mangalore,mumbai,paradip,port blair,tuticorin,vizag
from_port,,,,,,,,,,,,,,,,
chennai,0.0,30.0,1280.5,729.5,1835.0,1512.0,351.0,1866.5,928.0,925.0,1114.5,1506.5,578.0,844.0,760.5,334.0
ennore,30.0,0.0,929.0,867.0,1612.0,1206.0,351.0,1658.0,928.0,616.0,797.0,1206.0,721.0,794.0,349.0,419.0
goa,1280.5,929.0,0.0,1865.5,590.0,266.0,1279.0,622.0,96.0,383.0,190.5,261.0,1732.5,1613.0,633.0,1536.0
haldia,729.5,867.0,1865.5,0.0,2420.0,2097.0,517.0,2451.5,1795.0,1510.0,1699.5,2091.5,176.5,768.0,1345.5,419.0
jamnagar,1835.0,1612.0,590.0,2420.0,0.0,429.0,1963.0,49.0,684.0,939.0,744.5,424.0,2286.5,2297.0,1187.0,2090.5
jnpt,1512.0,1206.0,266.0,2097.0,429.0,0.0,1557.0,461.0,278.0,614.5,421.5,5.0,1963.5,1890.0,864.0,1767.5
kakinada,351.0,350.0,1297.0,517.0,1962.0,1556.0,0.0,2008.0,1278.0,967.0,1148.0,1556.0,370.0,913.0,699.0,74.0
kandla,1866.5,1658.0,622.0,2451.5,49.0,461.0,2009.0,0.0,718.0,971.0,776.5,456.0,2318.5,2140.0,1219.0,2122.0
karwar,1279.0,928.0,96.0,1796.0,684.0,278.0,1279.0,718.0,0.0,333.0,96.0,278.0,1649.0,1612.0,579.0,1347.0


In [22]:
new_dist_mtx = sub_dist_mtx.reset_index().melt(id_vars = 'from_port')

new_dist_mtx['from_port'].unique() 

port_name_dict = dict(zip(new_dist_mtx['from_port'].unique()  , [i + 1for i in range(len(new_dist_mtx['from_port'].unique() ))]))

def port_code_func(x):
    try:
        return port_name_dict[x]
    except:
        return x

new_dist_mtx = new_dist_mtx.applymap(port_code_func)

new_dist_mtx.head(2)

or_dist_mtx = np.array(new_dist_mtx.pivot_table(index = 'from_port' , columns = 'variable' , values = 'value'))

In [23]:
or_dist_mtx

array([[   0. ,   30. , 1280.5,  729.5, 1835. , 1512. ,  351. , 1866.5,
         928. ,  925. , 1114.5, 1506.5,  578. ,  844. ,  760.5,  334. ],
       [  30. ,    0. ,  929. ,  867. , 1612. , 1206. ,  351. , 1658. ,
         928. ,  616. ,  797. , 1206. ,  721. ,  794. ,  349. ,  419. ],
       [1280.5,  929. ,    0. , 1865.5,  590. ,  266. , 1279. ,  622. ,
          96. ,  383. ,  190.5,  261. , 1732.5, 1613. ,  633. , 1536. ],
       [ 729.5,  867. , 1865.5,    0. , 2420. , 2097. ,  517. , 2451.5,
        1795. , 1510. , 1699.5, 2091.5,  176.5,  768. , 1345.5,  419. ],
       [1835. , 1612. ,  590. , 2420. ,    0. ,  429. , 1963. ,   49. ,
         684. ,  939. ,  744.5,  424. , 2286.5, 2297. , 1187. , 2090.5],
       [1512. , 1206. ,  266. , 2097. ,  429. ,    0. , 1557. ,  461. ,
         278. ,  614.5,  421.5,    5. , 1963.5, 1890. ,  864. , 1767.5],
       [ 351. ,  350. , 1297. ,  517. , 1962. , 1556. ,    0. , 2008. ,
        1278. ,  967. , 1148. , 1556. ,  370. ,  913. ,  6

In [24]:
for i in set(port_name_dict.keys()).difference(set(pd_df_copy_net_all['port'].unique())):
    pd_df_copy_net_all.loc[len(pd_df_copy_net_all)] = [i , 0]


pd_df_copy_net_all = pd_df_copy_net_all.applymap(port_code_func).sort_values(by = 'port')
# demand_net_all= pd_df_copy_net_all[prodt].tolist()

In [25]:
pd_df_copy_net_all

Product,port,HFHSD,HSD,MS,SKO
0,1,-3.142570,37.700223,-7.296305,0.000000
1,2,0.000000,-37.700223,-18.229813,0.000000
2,3,-2.095043,-11.594080,-3.706210,-0.041220
3,4,11.575040,0.000000,19.460816,8.394744
4,5,0.000000,0.000000,0.110670,0.000000
5,6,0.000000,-12.687256,-2.328996,0.000000
6,7,-2.618807,0.000000,0.000000,0.000000
7,8,-0.838030,41.940287,-0.110670,0.000000
8,9,-5.237520,0.000000,0.000000,0.000000
9,10,-6.285140,-26.177182,-4.190838,0.000000


In [26]:
# for i in set(port_name_dict.keys()).difference(set(pd_df_copy_net_hsd['port'].unique())):
#     pd_df_copy_net_hsd.loc[len(pd_df_copy_net_hsd)] = [i , 0]


# pd_df_copy_net_hsd = pd_df_copy_net_hsd.applymap(port_code_func).sort_values(by = 'port')
# demand_net_hsd = pd_df_copy_net_hsd[prodt].tolist()

# prodt = 'MS'

# for i in set(port_name_dict.keys()).difference(set(pd_df_copy_net_ms['port'].unique())):
#     pd_df_copy_net_ms.loc[len(pd_df_copy_net_ms)] = [i , 0]


# pd_df_copy_net_ms = pd_df_copy_net_ms.applymap(port_code_func).sort_values(by = 'port')
# demand_net_ms = pd_df_copy_net_ms[prodt].tolist()





# demand_net_dict = dict(zip(['MS' , 'HSD'] , [list(pd_df_copy_net_ms['MS'].values) , list(pd_df_copy_net_hsd['HSD'].values)]))

# demand_net_dict

In [27]:
draft_df= pd.read_excel('/kaggle/input/data-to-play/vessel_wise_drafts.xlsx')
draft_dict = {'GP' : draft_df['GP'].values.tolist() , 'MR' : draft_df['MR'].values.tolist() , 'PR' : draft_df['PR'].values.tolist()  }

In [28]:
# draft_df = pd.read_excel("draft_restrictions_vho_MR.xlsx")

 

# draft_df['port'] = draft_df['Port'].apply(lambda x: x.strip().lower())

# draft_df = draft_df[draft_df['port'].isin(port_name_dict.keys())]

# draft_df.drop(columns = 'Port' , inplace = True)

# draft_comp_port_lst = set(port_name_dict.keys()).difference(set(draft_df['port'].unique()))

# draft_df = draft_df.reset_index(drop = True)

# for i in list(draft_comp_port_lst):

#     print(i)

#     draft_df.loc[len(draft_df)]  =     [200 , i]

   

# draft_df['index'] = draft_df['port'].apply(lambda x: port_name_dict[x] )

# draft_df = draft_df.sort_values(by = ['index'] )

 

# max_draft_dict = dict(zip(range(len(draft_df)) , draft_df['draft']))



In [29]:
# demand_net_hsd  = [38, -38, -12, 0, 0, -13, 0, 43, 0, -27, 22, -3, 10, -8, 0, -12]
# demand_net_ms = [-9, -21, -4, 23, 0, -3, 0, 0, 0, -5, 25, -2, -1, -2, 0, -1]

In [30]:
# demand_net_dict = dict(zip(['MS' , 'HSD'] , [demand_net_ms , demand_net_hsd]))

In [31]:
# demands_net = pd.DataFrame(demand_net_dict)

In [32]:
pd_df_copy_net_all.drop(columns = ['port'] , inplace = True)

In [33]:
pd_df_copy_net_all

Product,HFHSD,HSD,MS,SKO
0,-3.142570,37.700223,-7.296305,0.000000
1,0.000000,-37.700223,-18.229813,0.000000
2,-2.095043,-11.594080,-3.706210,-0.041220
3,11.575040,0.000000,19.460816,8.394744
4,0.000000,0.000000,0.110670,0.000000
5,0.000000,-12.687256,-2.328996,0.000000
6,-2.618807,0.000000,0.000000,0.000000
7,-0.838030,41.940287,-0.110670,0.000000
8,-5.237520,0.000000,0.000000,0.000000
9,-6.285140,-26.177182,-4.190838,0.000000


In [34]:

def evaluate_route(route, distance_matrix , vt):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    # Return to the starting port
    total_distance += distance_matrix[route[-1], route[0]]
    return (total_distance  * vessel_dict[vt][1])  + vessel_dict[vt][0]   ######### Adding variable and fixed cost

def evaluate_time(route, distance_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    # Return to the starting port
    total_distance += distance_matrix[route[-1], route[0]]
    return total_distance 




In [35]:
vessel_type_lst = ['MR' , 'GP' , 'PR']
#### All costs are in lakhs 
#### Capacities are in TMT 

fxt_cost_lst = [ 443  ,  362  ,  198] 
var_cost_lst = [12.0, 9.7, 5.3] 
# capacity_lst = [44 , 32 , 8]  ### We are reducing our capacity to be perfectly divisible by 4 ## Actual capacity is [47 , 35 , 10]
capacity_lst = [47 , 35 , 10]

# VOYAGE CHARTER COST = 15 LAKHS
vc_cost_per_mt = 1000 

### Avg COst per MT as per TC cost Summary shared is 14.7  


vessel_dict = dict(zip(vessel_type_lst,[(i,j , k) for i , j , k in zip(fxt_cost_lst ,var_cost_lst  , capacity_lst)]))

draft_dict 
demands_net = pd_df_copy_net_all 

In [36]:
def evaluate_cost(route, distance_matrix , vt):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    # Return to the starting port
    total_distance += distance_matrix[route[-1], route[0]]
    return (total_distance  * vessel_dict[vt][1])   + vessel_dict[vt][0]   ######### Adding variable and fixed cost

def evaluate_time(route, distance_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    # Return to the starting port
    total_distance += distance_matrix[route[-1], route[0]]
    return total_distance 


def flat_action_lst(nested_list):
    import itertools
    flattened_list = list(itertools.chain.from_iterable(
    sublist if isinstance(sublist, list) else [sublist] for sublist in nested_list))

    return flattened_list


'''Vessel optimizatin flow
Steps:


1. Select No. of ships / can be fixed at 4 initially  -------- before loop 1 

loop1 : for ship in no_of_ships
2. Trial.suggest_categorical for vessel type          -------- Bt loop 1 and loop 2
3. trial.suggest_int for len of route.                -------- Bt loop 1 and loop 2

loop2 : for route in len of route
4. Container_structure will be triggered to form a ship.
    container func : suggest start port.
                     suggest end port.
                     
                     suggest initial container allocation.
                     suggest initial product quantity.
        
5. after port and quantity is decided, correct_ship_dummy will trigger to rationalise the containers.

6.1  check if containers could be discharged based on available product in container for all containers
6.2 then run the container correction function again after discharging.
6.3 Check for available products , trial.suggest quantty and product to be picked.



need a function whick will take port and vessel to allocate products and quantities

additional penalties: Cleaning time , load_rate and dscharge rate as per vessel 

minimize : total cost of voyage/ metric tonn  , penalties , complete linkage


Take care of :

Positive load : will pick and grop according to capacity 

Draft break will be penalised with high penalty.
Link_non_completion will be penaised highly. 



Open points :

Deciding on structure of output and storage ---- Pickup and delivery actions have been recored 

Cleaning cost have been incorporated in container correctionn function, one loop hole is if the same product is refilled then anyways ceaning codst will be added. 



functions : 
    
    1. container_correction : re-adjust containers within ship   ----------- DONE 
    2. container_pickups : check the port and existing container structure , suggest product and quantities to be picked up. ----- DONE
    3. container_drop : check the port and existing container structure to discharge products. ----- done 
    4. container_structure : to initialize container structure.  ----- done
    
    cost per metric tonn function 
'''
    





class ShipLoadIsWrong(Exception):
    pass
def ship_load(containers):
    # try: 
    total_ship_load = sum([qty for product, qty in containers.values() if product != 'No_product'])
    # finally: 
    #     raise ShipLoadIsWrong("The ship load calculation is wrong.at decide pickup")
    return total_ship_load




import optuna
import copy
def decide_pickups(trial , containers, demands  , port_number , vt , ship_no , port , next_port):
    """ 
    Decides the quantity of each product to pick up at the next port based on existing container utilization
    and the available demands. Uses Optuna's suggest_int to determine the pickup quantities.

    Args:
    trial (optuna.trial.Trial): Optuna trial object to suggest pickup quantities.
    containers (dict): A dictionary with container IDs as keys and tuples (product, quantity) as values.
    demands (dict): A dictionary with product names as keys and quantities as values (positive for pickup, negative for drop).

    Returns:
    dict: A dictionary with container IDs as keys and tuples (product, new_quantity) as values after the pickups.
    """


    # Copy of the current container status to update 
    new_containers =  copy.deepcopy(containers) 
    
    action = []
    # Check existing products in containers
    existing_products = {product for product, qty in new_containers.values() if product != 'No_product'}
    
    # Iterate through each product in the demands and first fill the existing containers
    for product, demand_quantity in demands.items():
        if demand_quantity > 0:  # Pickup case
            remaining_demand = demand_quantity
            
            # First try to fill the existing containers with the same product
            for container_id, (current_product, current_quantity) in new_containers.items():
                total_ship_load = ship_load(new_containers)
                
                if current_product == product and current_quantity < vessel_dict[vt][2] / 4 :
                    # Suggest how much to pick up for this container
                    max_possible_pickup = max(0,min(vessel_dict[vt][2] / 4 - current_quantity, remaining_demand, draft_dict[vt][port]-total_ship_load , draft_dict[vt][next_port]-total_ship_load ))
                    
                    try:
                        # suggested_pickup = trial.suggest_float(f"pickup_{container_id}_{product}_{ship_no}_{port_number}_suggested_quantity_{port}", 0, max_possible_pickup)
                        suggested_pickup = max_possible_pickup 
                    except:
                        suggested_pickup = 0
                        # print('Pickup suggestion failed at ' +  f"pickup_{container_id}_{product}_{ship_no}_{port_number}")
                    
                    # print('Existing Product' , container_id , current_product , suggested_pickup )
                    # Update the container with the suggested pickup
                    new_quantity = current_quantity + suggested_pickup
                    new_containers[container_id] = (product, new_quantity)
                    
                    if suggested_pickup > 0:
                        action.append(('Pickup' ,container_id, product , suggested_pickup ))
                    else :
                        action.append(('No Action' ,container_id, product , suggested_pickup )) 
                        
                    # Decrease the remaining demand
                    remaining_demand -= suggested_pickup
                    # total_ship_load+=suggested_pickup
                    # Adjust the demands dictionary
                    demands[product] -= suggested_pickup 
                    
                    
                # # If the remaining demand is fulfilled, break the loop
                # if remaining_demand == 0:
                #     break 

    # Now handle the empty containers by suggesting new products
    for container_id, (current_product, current_quantity) in new_containers.items():
        if current_product == 'No_product':  # This is an empty container
            # Suggest a product from the demands that has a positive pickup quantity
            available_products = list(demands_net.columns)  # [product for product, qty in demands.items() if qty > 0]
            if available_products:
                # Suggest which product to load into the empty container
                total_ship_load  = ship_load(new_containers)
                
                if port_number != 0: 
                    suggested_product = trial.suggest_categorical(f"pickup_{container_id}_{product}_{ship_no}_{port_number}_suggested_product_{port}", available_products)

                    max_possible_pickup = max(min(vessel_dict[vt][2] / 4, demands[suggested_product] , draft_dict[vt][port] - total_ship_load , draft_dict[vt][next_port] - total_ship_load ) , 0 )

                    # suggested_pickup = trial.suggest_float(f"pickup_{container_id}_{suggested_product}", 0, max_possible_pickup)
                    suggested_pickup = max_possible_pickup
                    
                    # print('New Container Product' , container_id , current_product , suggested_product, suggested_pickup  )
                    # Update the container with the suggested product and quantity
                    
                    if suggested_pickup > 0:
                        action.append(('Pickup' ,container_id, suggested_product , suggested_pickup ))
                        new_containers[container_id] = (suggested_product, suggested_pickup)
                    else :
                        action.append(('No Action',container_id , current_product , suggested_pickup ))
                        new_containers[container_id] = (current_product, suggested_pickup)
                        
                        
                    # Reduce the remaining demand for the picked product
                    demands[suggested_product] -= suggested_pickup
                    
                else: ##### Pick up everything at starting port 
                    
                    suggested_product = trial.suggest_categorical(f"pickup_{container_id}_{product}_{ship_no}_{port_number}_suggested_product_{port}", available_products)

                    max_possible_pickup = max(0 , min(vessel_dict[vt][2] / 4, demands[suggested_product] , draft_dict[vt][port]-total_ship_load  , draft_dict[vt][next_port]-total_ship_load ))

                    suggested_pickup = max_possible_pickup
                    
                    # Update the container with the suggested product and quantity
                    # new_containers[container_id] = (suggested_product, suggested_pickup)
                    
                    if suggested_pickup >0:
                        action.append(('Pickup' ,container_id, suggested_product , suggested_pickup ))
                        new_containers[container_id] = (suggested_product, suggested_pickup)
                    else :
                        action.append(('No Action' ,container_id, current_product , suggested_pickup ))
                        new_containers[container_id] = (current_product, suggested_pickup)
                        
                    # Reduce the remaining demand for the picked product
                    demands[suggested_product] -= suggested_pickup 

    return new_containers , demands , action  


def container_correction(containers , vt):
#### Counting empty containers to calculate the cleaning cost

    empty_cont_b = len([product for container, (product, quantity) in containers.items() if product == 'No_product'])

    new_containers = {}
    for container, (product, quantity) in containers.items():
              
        if quantity == 0 :
            new_containers[container] = ('No_product' , 0)
        else:
            new_containers[container] = (product , quantity) 

    empty_cont_a = len([product for container, (product, quantity) in new_containers.items() if product == 'No_product'])

    #### Containers cleaned

    cleaned_containers = empty_cont_a - empty_cont_b 

    return new_containers , cleaned_containers


def decide_discharges(trial , containers, demands  , port_number , vt , ship_no ): 
    """
    Decides the quantity of each product to drop off at the next port based on existing container utilization
    and the available demands. Uses Optuna's suggest_int to determine the discharge quantities.

    Args:
    trial (optuna.trial.Trial): Optuna trial object to suggest discharge quantities.
    containers (dict): A dictionary with container IDs as keys and tuples (product, quantity) as values.
    demands (dict): A dictionary with product names as keys and quantities as values (positive for pickup, negative for drop).

    Returns:
    dict: A dictionary with container IDs as keys and tuples (product, new_quantity) as values after the discharges.
    """
    ### Initializing a dictionary to kep track of drops 
    drop_dict = {} 
    for i in dict(demands).keys() :
        drop_dict[i] = 0
    
    
    # Copy of the current container status to update
    new_containers = copy.deepcopy(containers)
    # arrange containers in ascending order of quantity
    new_containers = dict(sorted(new_containers.items(), key=lambda item: item[1][1]))
    
    action = []
    # Iterate through each product in the demands
    for product, demand_quantity in demands.items():
        if demand_quantity < 0:  # Drop-off case
            remaining_demand = abs(demand_quantity)
            
            # Try to drop off the product from containers that have the product
            for container_id, (current_product, current_quantity) in new_containers.items():
                if current_product == product and current_quantity > 0:
                    # Suggest how much to drop off from this container
                    max_possible_discharge = max(0 , min(current_quantity, remaining_demand ))
                    # suggested_discharge = trial.suggest_int(f"discharge_{container_id}_{product}_{ship_no}_{port_number}", 0, max_possible_discharge)
                    suggested_discharge = max_possible_discharge
                    # Update the container with the suggested discharge
                    new_quantity = current_quantity - suggested_discharge
                    new_containers[container_id] = (current_product, new_quantity)
                    
                    #### Recording Delivery actions #### 
                    if suggested_discharge > 0:
                        action.append(('Delivery' , container_id,  current_product , suggested_discharge ))
                    else :
                        action.append(('No Action' ,container_id , current_product , suggested_discharge ))
                        
                    
                    # Decrease the remaining demand
                    remaining_demand -= suggested_discharge 
                    
                    # Adjust the demands dictionary
                    demands[product] += suggested_discharge
                    
                    # Adding drops to the drop_dict
                    drop_dict[product] += suggested_discharge
                # If the remaining demand is fulfilled, break the loop
                if remaining_demand == 0:
                    break

    return new_containers , demands , action , drop_dict 


def cont_utilization(containers , vt):
    total_qty = 0
    for cont, (prod , qty) in containers.items():
        total_qty += qty
    under_util_perc = 100 - ((total_qty/vessel_dict[vt][2]) * 100)
    return under_util_perc 


In [37]:
vessel_dict

{'MR': (443, 12.0, 47), 'GP': (362, 9.7, 35), 'PR': (198, 5.3, 10)}

In [38]:
import optuna
from optuna.samplers import TPESampler

# Custom TPESampler class with a decaying gamma
class CustomTPESampler(TPESampler):
    def __init__(self, decay_rate=0.9, min_gamma=0.01, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.decay_rate = decay_rate  # Rate of decay for gamma
        self.min_gamma = min_gamma    # Minimum value for gamma

    # Override the _gamma function to apply custom decay
    def _gamma(self, trials: list):
        n = len(trials)
        
        # Apply decay rate to gamma, gradually decreasing it
        gamma = min(5, n) * (self.decay_rate ** n)
        
        
        
        # Ensure gamma does not fall below a minimum threshold
        return max(self.min_gamma, 1.0 / gamma) 


# sampler = CustomTPESampler(decay_rate=0.999, min_gamma=0.02)  

In [39]:
import optuna
import numpy as np
from joblib import Parallel, delayed
import csv
from optuna.storages import RDBStorage 

In [40]:
min_ships = 7
max_ships = 11

distance_matrix = or_dist_mtx / (12 * 24)
# sel_port_lst = [0 ,1,2,3,5,7,9,10,11,12,13,15]  
output_file = 'Multiproduct_Full_discharge_single_obj__all_route_all_vt_45_days_only_penalty_all_pickup_tuned_parameter_5000_warmup_31_days_single_core.csv'
no_trials = 100

def objective(trial):
    
    count_port_visit  = 10
    ### Time take for picking up and drop
    pd_time_all = 0 
    
    ### Container cleaning instances 
    total_cleaning_cost =  0
    
    ### Container Underutilization percentage  
    total_under_util_penalty = 0
    ### keep track of total distance in days 
    total_distance = 0
    total_distance_opt = 0
    ### Draft break , incomplete linkage penalty 
    total_penalty = 0
    
    ### Fixed and variable ship cost
    total_cost = 0 
    
    ### Skipping ports penalty 
    linkage_penalty = 0
    all_routes = []

    demands = demands_net.copy(deep = True)
    product_list = list(demands.keys())
    
    ### Dictionary to track dropped quantaties
    ship_drop_dict = {}
    for i in product_list:
        ship_drop_dict[i] = 0
    
    

    
    num_ships = trial.suggest_int('num_ships' , min_ships,max_ships) 
    ship_type=[]
    startport_per_ship_type=[]
    product_selection_per_type=[]
    route_length_per_type=[]
    
    small_ship_count = 0
    for ship in range(num_ships):
      #### Selecting ship type
        vt = trial.suggest_categorical(f'{ship}_selected_type_' , vessel_type_lst)
        ship_type.append(vt)
        #### Suggesting route length 
        route_length = trial.suggest_int('route_length' , 1 , count_port_visit )
        route_length_per_type.append(route_length)
        product_selection=trial.suggest_categorical(f'starting_list_{ship}',product_list)
        product_selection_per_type.append(product_selection)

        start_port = trial.suggest_categorical(f'ship_{ship}_start_port_{product_selection}', list(demands_net[demands_net[product_selection]>0].index))
        startport_per_ship_type.append(start_port)
    for ship in range(num_ships):
        
        #### Selecting ship type
        vt = ship_type[ship]
        #### Suggesting route length 
        route_length =route_length_per_type[ship]
        
        pd_time = 0
        product_selection=product_selection_per_type[ship]
        
        start_port = startport_per_ship_type[ship]
        # end_port =  trial.suggest_categorical(f'ship_{ship}_end_port_hsd',end_port_lst_hsd,list(demands_net[demands_net[product_selection]<0].index))
        
        #### Creating whole route before quantity 
        route = [start_port]
        
        for port_i in range(route_length):
            port = trial.suggest_int(f'ship_{ship}_port_{port_i}', 0 , 15) ## 21 Aug : Suggesting from selected ports only 
            route.append(port)
        route.append(start_port)    
        
        ### Tracking the no of times containers are cleaned 
        ship_cleaning_count = 0
        # all_action = []
        ship_loads = []
        counter = 0 
        draft_break = 0
        ship_under_util_penalty = 0
        
        for port_number in range(len(route)): ### Iterating over whole route suggested before 
            
            # action_discharge
            all_action = [] 
            all_containers = []
            
            if port_number !=0 :
                
                port = route[port_number]
                
                if port_number == len(route) -1  :
                    next_port = port
                else: 
                    next_port = route[port_number + 1] 
                    
                # port = trial.suggest_int(f'ship_{ship}_port_{i}', 0, num_ports - 1)
#                 port = trial.suggest_int(f'ship_{ship}_port_{port_number}', 0 , 15) ## 21 Aug : Suggesting from selected ports only 
#                 route.append(port)
                
                # Storing container status while entering the ports 
                all_containers.append(('While_entering' , containers , port_number , ship   ) )
                ### Checking for draft break while entering 
                if abs(min(draft_dict[vt][port] - sum([qty for product, qty in containers.values()] ), 0))>0:
                    total_penalty += abs(min(draft_dict[vt][port] - sum([qty for product, qty in containers.values()] ), 0))*1e+4
                else:
                    total_penalty += 0                                                         
                                                         
                ### First discharging the available products if drop is available
                containers , demands_slice , action_delivery , drop_dict  =  decide_discharges(trial , containers, dict(demands.loc[port])  , port_number , vt , ship )
                all_action.append(action_delivery)
                
                ### Adding droped product quantaties to ship_drop_dict
                for i in drop_dict.keys():
                    ship_drop_dict[i] += drop_dict[i] 
                
                
                ##### Updating the demands after dropping 
                demands.loc[port] = list(dict(demands_slice).values()) 
                
                ### re-adjusting the containers after discharge
                containers , cleaned_containers =  container_correction(containers , vt)
                ship_cleaning_count += cleaned_containers
                
                
                ### Picking up products if available                         trial , containers, demands  , port_number , vt , ship_no , port
                containers , demands_slice , action_pickup  = decide_pickups(trial , containers, dict(demands.loc[port])  , port_number , vt , ship , port , next_port)
                all_action.append(action_pickup)
                ##### Updating the demands
                demands.loc[port] = list(dict(demands_slice).values())
                
                ### re-adjusting the containers after pickup
                containers , cleaned_containers =  container_correction(containers , vt)
                ship_cleaning_count += cleaned_containers 

                ### Checking for draft break while exiting port 
                if abs(min(draft_dict[vt][port] - sum([qty for product, qty in containers.values()] ), 0))>0:
                    total_penalty += abs(min(draft_dict[vt][port] - sum([qty for product, qty in containers.values()] ), 0))*1e+4
                else:
                    total_penalty += 0
                    
                all_containers.append(('While_exiting' , containers , port_number , ship   ) )                                                      
                #### IF IT IS A STARING PORT #### goto ELSE:
            else:  
                                     
                port = start_port
                next_port = port
                ### Initializing the containers 
                containers = {}
                for cont in range(1 , 4 + 1):
                    containers['C' + str(cont)] = ('No_product' , 0)
                
                all_containers.append(('At_First_port' , containers , port_number , ship   ) )
                ### Picking up products if available
                containers , demands_slice , action_pickup  = decide_pickups(trial , containers, dict(demands.loc[port])  , port_number , vt , ship , port , next_port)   ####### Returning pickupfunc output
                all_action.append(action_pickup) 
                                     
                ##### Updating the demands after pickup 
                demands.loc[port] = list(dict(demands_slice).values())
                
                ### re-adjusting the containers after pickup
                containers , cleaned_containers =  container_correction(containers , vt) 
                ship_cleaning_count += cleaned_containers
                
                ### Checking for draft break while exiting port 
                if abs(min(draft_dict[vt][port] - sum([qty for product, qty in containers.values()] ), 0))>0:
                    total_penalty += abs(min(draft_dict[vt][port] - sum([qty for product, qty in containers.values()] ), 0))*1e+4
                else:
                    total_penalty += 0
                
                all_containers.append(('Exiting_first_port' , containers , port_number , ship  ) )
                
            
            ### Calculating if there has been a pickup or drop and adding the timer (pd_time )
            # print(all_action)
            if len([i[0] for i in flat_action_lst(all_action) if i[0] != 'No Action']) > 0:
                pd_time += 1.5
                                     
            ### Appending action                        
            ship_loads.append((all_action , all_containers))
            
            ### Adding port wise underutilization percentage
            ship_under_util_penalty += cont_utilization(containers , vt )
            
        # # Constraint violation check
        #     if load_ms < 0 or load_hsd < 0 :
        #         total_penalty +=100000000*abs((min(0,load_ms)+min(0,load_hsd))) 
            
        # Ensure the ship returns to the starting port
        # route.append(start_port)
        
        all_routes.append(( ship, vt , route, ship_loads)) 
        # print(all_routes)
        
        # Check if the net quantity is zero when the route ends 
        total_penalty += sum([qty for product, qty in containers.values()]) * vc_cost_per_mt * 1000 
        
        total_under_util_penalty += ship_under_util_penalty/route_length 
        
        # Calculate total distance and cleaning cost 
        total_cost += evaluate_cost(route, distance_matrix , vt ) 
        total_cleaning_cost += ship_cleaning_count * 0  ## (add cleaning cost factor)
        total_cost += pd_time * 0 #### Assuming port charges of 50K
        # Evalusting total time
        

        if (evaluate_time(route , distance_matrix) + pd_time > 31) :
            ship_time_cc = float('inf')
        else:
            ship_time_cc =   0 

        small_ship_count += 1 
        
        total_distance += ship_time_cc 
        total_distance_opt += evaluate_time(route , distance_matrix) + pd_time
                             
    # Ensure all demands are met                         
    total_penalty += demands.abs().sum().sum() * vc_cost_per_mt  * 1000     
    print('remaining Load' , demands.abs().sum().sum())
    # Ensure every port is visited
    
    visited_ports = []
    for i in range(len(all_routes)):
        visited_ports.append(all_routes[i][2])
    
    visited_ports = set(flat_action_lst(visited_ports)) 
    
    # visited_ports = set([[] + tp[2] for tp in all_routes])
    
    not_visit_port = len(set(demands_net.index).difference(set(demands_net[demands_net.eq(0).all(axis=1)].index)).difference(visited_ports))  
    linkage_penalty += not_visit_port * 10000
    # linkage_penalty = max(linkage_penalty , 20000)
    
    # Add total cost , calculate cost per metric tonn , total_penalty (draft , remaining_load , remaning_linkages)  , skipped_ports(linkage_penalty) , total_time ( tot_dist + pd_time)
    
    total_fixed_variable_cost = total_cost + total_cleaning_cost
    total_product_delivered = sum(list(ship_drop_dict.values()))
     
        
    cost_per_mt = total_fixed_variable_cost/(total_product_delivered+1e-7)
                             
    ### TOtal Time :
    total_time_days = total_distance  ##### pd_time_all if not adding while evaluating ship time 
      
    
    if total_distance_opt  > num_ships * 30:
        total_time_days += float('inf')
    else: 
        total_time_days += 0
        
    ### Total Penalty 
    total_penalty  
    linkage_penalty
    
    
    # total_obj = (cost_per_mt  ) + ( total_penalty)  + total_time_days 
    # total_obj = (total_fixed_variable_cost  ) + (total_penalty) + (total_time_days)  + (total_under_util_penalty * 2.5) + (linkage_penalty)  ###### Here not adding no of days in objective value 
                                                                                        #####, just adding if it is infinity , replaced time by 0 if time is not infinity
    total_obj =  (total_penalty) + (total_time_days)                        
                             
    ######### Printing trial jorney details and objectives 
    # Print out the journey details for this trial
    # print(f"Trial {trial.number} - Total Distance: {total_distance}, Total Penalty: {total_penalty}, Total Tonmiles: {total_tonmiles}")
    # for ship, route, loads in all_routes:
    #     print(f"  Ship {ship}: Route - {route}")
    #     for port, load_change, load, action in loads:
    #         print(f"    Port {port}: {action} {load_change} units (Current Load: {load})")
    
    
    # Save trial results (total_obj < 5e+8) and
    if  (total_obj != np.inf):
        trial_results.append({
            'trial_number': trial.number,
            'params': trial.params,
            'values': (cost_per_mt , total_distance_opt , total_penalty , linkage_penalty ,total_obj , total_under_util_penalty), 
            'routes': all_routes
        })

#     if total_distance < 25 :
#         total_distance +=  (25 - total_distance) * 10000 
    
    # Return all three objectives 
    if (total_obj == None) or (total_fixed_variable_cost == None):
        total_obj , total_fixed_variable_cost = float('inf') , float('inf')
        print("skip_condition")
    print(trial.number)
    return total_obj,total_fixed_variable_cost
optuna.logging.set_verbosity(optuna.logging.WARNING) 

# Define the parallelization function
def parallel_optimize(n_trials):
    storage = RDBStorage(url='sqlite:///Multiproduct_Full_discharge_single_obj__all_route_all_vt_45_days_only_penalty_all_pickup_tuned_parameter_5000_warmup_31_days_single_core.db')
    # pruner=optuna.pruners.SuccessiveHalvingPruner()
#     pruner = optuna.pruners.NopPruner()

    # pruner = optuna.pruners.HyperbandPruner( max_resource='auto') 
    study = optuna.create_study(
        directions=["minimize",'minimize'],
        sampler= optuna.samplers.TPESampler(n_startup_trials = 3500 , constant_liar = True , multivariate = True , group = True),
        storage = storage ,
        load_if_exists= True,
        study_name = 'Vessel_planning_tc'
#         pruner = pruner
    )

    study.optimize(objective, n_trials=n_trials, n_jobs = 1)# Use all iilable cores
    return study 

# List to store all trial results

for iterartions in range(10):
    trial_results = []
    print(iterartions)
    # Run optimization in parallel
    study = parallel_optimize(n_trials = no_trials)
    # if iterations>7:
    #### Check if the file exists to determine if the header needs to be written
    file_exists = os.path.isfile(output_file)

    ### Open the CSV file in append mode (here we are using "a" for append if file already exists)
    with open(output_file, 'a', newline='') as csvfile:
        fieldnames = ['trial_number', 'params', 'values', 'routes']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header only if the file is newly created
        if not file_exists:
            writer.writeheader()
        try:
            for result in trial_results:
                writer.writerow({
                    'trial_number': result['trial_number'],
                    'params': result['params'],
                    'values': result['values'],
                    'routes': result['routes']

                })
        except:
            continue
    from IPython.display import clear_output
    clear_output()
    print(f"Results saved to {output_file}")

Results saved to Multiproduct_Full_discharge_single_obj__all_route_all_vt_45_days_only_penalty_all_pickup_tuned_parameter_5000_warmup_31_days_single_core.csv


In [41]:
# def evaluate_route(route, distance_matrix , vt):
#     total_distance = 0
#     for i in range(len(route) - 1):
#         total_distance += distance_matrix[route[i], route[i + 1]]
#     # Return to the starting port
#     total_distance += distance_matrix[route[-1], route[0]]
#     return (total_distance  * vessel_dict[vt][1])  + vessel_dict[vt][0]   ######### Adding variable and fixed cost

# def evaluate_time(route, distance_matrix):
#     total_distance = 0
#     for i in range(len(route) - 1):
#         total_distance += distance_matrix[route[i], route[i + 1]]
#     # Return to the starting port
#     total_distance += distance_matrix[route[-1], route[0]]
#     return total_distance 




# def flat_action_lst(nested_list):
#     import itertools
#     flattened_list = list(itertools.chain.from_iterable(
#     sublist if isinstance(sublist, list) else [sublist] for sublist in nested_list))

#     return flattened_list


# '''Vessel optimizatin flow
# Steps:


# 1. Select No. of ships / can be fixed at 4 initially  -------- before loop 1 

# loop1 : for ship in no_of_ships
# 2. Trial.suggest_categorical for vessel type          -------- Bt loop 1 and loop 2
# 3. trial.suggest_int for len of route.                -------- Bt loop 1 and loop 2

# loop2 : for route in len of route
# 4. Container_structure will be triggered to form a ship.
#     container func : suggest start port.
#                      suggest end port.
                     
#                      suggest initial container allocation.
#                      suggest initial product quantity.
        
# 5. after port and quantity is decided, correct_ship_dummy will trigger to rationalise the containers.

# 6.1  check if containers could be discharged based on available product in container for all containers
# 6.2 then run the container correction function again after discharging.
# 6.3 Check for available products , trial.suggest quantty and product to be picked.



# need a function whick will take port and vessel to allocate products and quantities

# additional penalties: Cleaning time , load_rate and dscharge rate as per vessel 

# minimize : total cost of voyage/ metric tonn  , penalties , complete linkage


# Take care of :

# Positive load : will pick and grop according to capacity 

# Draft break will be penalised with high penalty.
# Link_non_completion will be penaised highly. 



# Open points :

# Deciding on structure of output and storage ---- Pickup and delivery actions have been recored 

# Cleaning cost have been incorporated in container correctionn function, one loop hole is if the same product is refilled then anyways ceaning codst will be added. 



# functions : 
    
#     1. container_correction : re-adjust containers within ship   ----------- DONE 
#     2. container_pickups : check the port and existing container structure , suggest product and quantities to be picked up. ----- DONE
#     3. container_drop : check the port and existing container structure to discharge products. ----- done 
#     4. container_structure : to initialize container structure.  ----- done
    
#     cost per metric tonn function 
# '''
    





# class ShipLoadIsWrong(Exception):
#     pass
# def ship_load(containers):
#     # try: 
#     total_ship_load = sum([qty for product, qty in containers.values() if product != 'No_product'])
#     # finally: 
#     #     raise ShipLoadIsWrong("The ship load calculation is wrong.at decide pickup")
#     return total_ship_load




# import optuna

# def decide_pickups(trial , containers, demands  , port_number , vt , ship_no , port):
#     """ 
#     Decides the quantity of each product to pick up at the next port based on existing container utilization
#     and the available demands. Uses Optuna's suggest_int to determine the pickup quantities.

#     Args:
#     trial (optuna.trial.Trial): Optuna trial object to suggest pickup quantities.
#     containers (dict): A dictionary with container IDs as keys and tuples (product, quantity) as values.
#     demands (dict): A dictionary with product names as keys and quantities as values (positive for pickup, negative for drop).

#     Returns:
#     dict: A dictionary with container IDs as keys and tuples (product, new_quantity) as values after the pickups.
#     """
    
#     # Copy of the current container status to update 
#     new_containers = containers.copy()
#     action = []
#     # Check existing products in containers 
#     existing_products = {product for product, qty in containers.values() if product != 'No_product'}
    
#     # Iterate through each product in the demands and first fill the existing containers
#     for product, demand_quantity in demands.items():
#         if demand_quantity > 0:  # Pickup case
#             remaining_demand = demand_quantity
            
#             # First try to fill the existing containers with the same product
#             for container_id, (current_product, current_quantity) in new_containers.items():
#                 total_ship_load = ship_load(new_containers)
                
#                 if current_product == product and current_quantity < vessel_dict[vt][2] / 4 :
#                     # Suggest how much to pick up for this container
#                     max_possible_pickup = max(0,min(vessel_dict[vt][2] / 4 - current_quantity, remaining_demand, draft_dict[vt][port]-total_ship_load))
                    
#                     try:
#                         suggested_pickup = trial.suggest_int(f"pickup_{container_id}_{product}_{ship_no}_{port_number}_suggested_quantity_{port}", 0, max_possible_pickup)
#                     except:
#                         suggested_pickup = 0
#                         print('Pickup suggestion failed at ' +  f"pickup_{container_id}_{product}_{ship_no}_{port_number}")
                        
#                     # Update the container with the suggested pickup
#                     new_quantity = current_quantity + suggested_pickup
#                     new_containers[container_id] = (product, new_quantity)
                    
#                     if suggested_pickup > 0:
#                         action.append(('Pickup' ,container_id, product , suggested_pickup ))
#                     else :
#                         action.append(('No Action' ,container_id, product , suggested_pickup )) 
                        
#                     # Decrease the remaining demand
#                     remaining_demand -= suggested_pickup
#                     # total_ship_load+=suggested_pickup
#                     # Adjust the demands dictionary
#                     demands[product] -= suggested_pickup 
                    
                    
#                 # # If the remaining demand is fulfilled, break the loop
#                 # if remaining_demand == 0:
#                 #     break 

#     # Now handle the empty containers by suggesting new products
#     for container_id, (current_product, current_quantity) in new_containers.items():
#         if current_product == 'No_product':  # This is an empty container
#             # Suggest a product from the demands that has a positive pickup quantity
#             available_products = list(demands_net.cols)  # [product for product, qty in demands.items() if qty > 0]
#             if available_products:
#                 # Suggest which product to load into the empty container
#                 total_ship_load  = ship_load(new_containers)
                
#                 if port_number != 0: 
#                     suggested_product = trial.suggest_categorical(f"pickup_{container_id}_{product}_{ship_no}_{port_number}_suggested_product_{port}", available_products)

#                     max_possible_pickup = max(min(vessel_dict[vt][2] / 4, demands[suggested_product] , draft_dict[vt][port] - total_ship_load ) , 0 )

#                     suggested_pickup = trial.suggest_int(f"pickup_{container_id}_{suggested_product}", 0, max_possible_pickup)

#                     # Update the container with the suggested product and quantity
#                     new_containers[container_id] = (suggested_product, suggested_pickup)
#                     if suggested_pickup >0:
#                         action.append(('Pickup' ,container_id, product , suggested_pickup ))
#                     else :
#                         action.append(('No Action',container_id , product , suggested_pickup ))
                        
                        
#                     # Reduce the remaining demand for the picked product
#                     demands[suggested_product] -= suggested_pickup
                    
#                 else: ##### Pick up everything at starting port 
                    
#                     suggested_product = trial.suggest_categorical(f"pickup_{container_id}_{product}_{ship_no}_{port_number}_suggested_product_{port}", available_products)

#                     max_possible_pickup = max(0 , min(vessel_dict[vt][2] / 4, demands[suggested_product] , draft_dict[vt][port]-total_ship_load ))

#                     suggested_pickup = max_possible_pickup
                    
#                     # Update the container with the suggested product and quantity
#                     new_containers[container_id] = (suggested_product, suggested_pickup)
                    
#                     if suggested_pickup >0:
#                         action.append(('Pickup' ,container_id, product , suggested_pickup ))
#                     else :
#                         action.append(('No Action' ,container_id, product , suggested_pickup ))
                        
#                     # Reduce the remaining demand for the picked product
#                     demands[suggested_product] -= suggested_pickup 

#     return new_containers , demands , action 

# # Example usage with Optuna 

# # def optuna_objective(trial):
# #     containers = {
# #         'Container_1': ('Oil_A', 3),
# #         'Container_2': ('Oil_A', 2),
# #         'Container_3': ('No_product', 0),
# #         'Container_4': ('Oil_B', 1)
# #     }
# #     demands = {
# #         'Oil_A': 7,
# #         'Oil_C': 4
# #     }
    
# #     new_containers,demands,actions = decide_pickups(trial, containers,demands, port_number=1,vt='MR',ship_no=1 , port = 1)
# #     print(new_containers)
    
# #     # You can add logic here to compute a score for Optuna based on some objective criteria
# #     return 1  # Dummy return value

# # # Run Optuna optimization
# # study = optuna.create_study(direction='maximize')
# # study.optimize(optuna_objective, n_trials=1) 




# list(demands_net.columns)




# def container_correction(containers , vt):
# #### Counting empty containers to calculate the cleaning cost

#     empty_cont_b = len([product for container, (product, quantity) in containers.items() if product == 'No_product'])

#     new_containers = {}
#     for container, (product, quantity) in containers.items():
              
#         if quantity == 0 :
#             new_containers[container] = ('No_product' , 0)
#         else:
#             new_containers[container] = (product , quantity)

#     empty_cont_a = len([product for container, (product, quantity) in new_containers.items() if product == 'No_product'])

#     #### Containers cleaned

#     cleaned_containers = empty_cont_a - empty_cont_b 

#     return new_containers , cleaned_containers


# # containers = {
# #     'Container_1': ('HSD', 0),
# #     'Container_2': ('MS', 4),
# #     'Container_3': ('HSD', 3),
# #     'Container_4': ('MS', 2)
# # }

# # corrected_containers , cl = container_correction(containers , 'MR')
# # print(corrected_containers , cl) 

# # def container_correction(containers , vt):
# #     """
# #     Corrects the container allocation by combining same products if possible.
    
# #     Args:
# #     containers (dict): A dictionary with container IDs as keys and tuples (product, quantity) as values.
    
# #     Returns:
# #     dict: Updated dictionary with corrected container allocation.
# #     """
    
# #     #### Counting empty containers to calculate the cleaning cost
# #     empty_cont_b = len([product for container, (product, quantity) in containers.items() if product == 'No_product'])

# #     new_containers = {}
# #     for container, (product, quantity) in containers.items():
# #         if quantity == 0 :
# #             new_containers[container] = ('No_product' , 0)
# #         else:
# #             new_containers[container] = (product , quantity)
    
    
    
    
# #     # Create a temporary dictionary to count the total quantity of each product
# #     product_quantities = {}
# #     action = []
    
    
    
# #     # Iterate through each container to accumulate quantities of the same product
# #     for container, (product, quantity) in containers.items():
# #         if product != 'No_product':
# #             if product in product_quantities:
# #                 product_quantities[product] += quantity
# #             else:
# #                 product_quantities[product] = quantity
    
# #     # Reset containers and index to keep track of the containers being filled
# #     container_ids = list(containers.keys())
# #     index = 0 
    
# #     # Assign combined quantities back to containers, respecting the max capacity of 10
# #     for product, total_quantity in product_quantities.items():
# #         while total_quantity > 0 and index < len(container_ids):
# #             if total_quantity > vessel_dict[vt][2] / 4:
# #                 containers[container_ids[index]] = (product, vessel_dict[vt][2] / 4)
# #                 total_quantity -= vessel_dict[vt][2] / 4
# #             else:
# #                 containers[container_ids[index]] = (product, total_quantity)
# #                 total_quantity = 0
# #             index += 1
    
# #     # Mark remaining containers as empty
# #     while index < len(container_ids):
# #         containers[container_ids[index]] = ('No_product', 0)
# #         index += 1 
    
# #     #### Counting Empty containers after making adjustments 
# #     empty_cont_a = len([product for container, (product, quantity) in containers.items() if product == 'No_product'])
    
# #     #### Containers cleaned
    
# #     cleaned_containers = empty_cont_a - empty_cont_b 
    
# #     return containers , cleaned_containers 

# # # # Example usage
# # containers = {
# #     'Container_1': ('HSD', 6),
# #     'Container_2': ('MS', 4),
# #     'Container_3': ('HSD', 3),
# #     'Container_4': ('MS', 2)
# # }

# # # corrected_containers , cl = container_correction(containers , 'MR')
# # # print(corrected_containers , cl) 



# import optuna

# def decide_discharges(trial , containers, demands  , port_number , vt , ship_no ): 
#     """
#     Decides the quantity of each product to drop off at the next port based on existing container utilization
#     and the available demands. Uses Optuna's suggest_int to determine the discharge quantities.

#     Args:
#     trial (optuna.trial.Trial): Optuna trial object to suggest discharge quantities.
#     containers (dict): A dictionary with container IDs as keys and tuples (product, quantity) as values.
#     demands (dict): A dictionary with product names as keys and quantities as values (positive for pickup, negative for drop).

#     Returns:
#     dict: A dictionary with container IDs as keys and tuples (product, new_quantity) as values after the discharges.
#     """
#     ### Initializing a dictionary to kep track of drops 
#     drop_dict = {} 
#     for i in dict(demands).keys() :
#         drop_dict[i] = 0
    
    
#     # Copy of the current container status to update
#     new_containers = containers.copy()
#     action = []
#     # Iterate through each product in the demands
#     for product, demand_quantity in demands.items():
#         if demand_quantity < 0:  # Drop-off case
#             remaining_demand = abs(demand_quantity)
            
#             # Try to drop off the product from containers that have the product
#             for container_id, (current_product, current_quantity) in new_containers.items():
#                 if current_product == product and current_quantity > 0:
#                     # Suggest how much to drop off from this container
#                     max_possible_discharge = max(0 , min(current_quantity, remaining_demand ))
#                     # suggested_discharge = trial.suggest_int(f"discharge_{container_id}_{product}_{ship_no}_{port_number}", 0, max_possible_discharge)
#                     suggested_discharge = max_possible_discharge
#                     # Update the container with the suggested discharge
#                     new_quantity = current_quantity - suggested_discharge
#                     new_containers[container_id] = (product, new_quantity)
                    
#                     #### Recording Delivery actions #### 
#                     if suggested_discharge > 0: 
#                         action.append(('Delivery' , container_id,product , suggested_discharge ))
#                     else :
#                         action.append(('No Action' ,container_id , product , suggested_discharge ))
                        
                    
#                     # Decrease the remaining demand
#                     remaining_demand -= suggested_discharge
                    
#                     # Adjust the demands dictionary
#                     demands[product] += suggested_discharge
                    
#                     # Adding drops to the drop_dict
#                     drop_dict[product] += suggested_discharge
#                 # If the remaining demand is fulfilled, break the loop
#                 if remaining_demand == 0:
#                     break

#     return new_containers , demands , action , drop_dict 

# # Example usage with Optuna
# # def optuna_objective(trial):
# #     containers = {
# #         'Container_1': ('Oil_A', 5),
# #         'Container_2': ('Oil_A', 4),
# #         'Container_3': ('Oil_B', 6),
# #         'Container_4': ('No_product', 0)
# #     }
# #     demands = {
# #         'Oil_A': -7,  # Indicates a drop-off of 7 units of Oil_A
# #         'Oil_B': -3   # Indicates a drop-off of 3 units of Oil_B
# #     }
    
# #     new_containers = decide_discharges(trial , containers, demands  , 1 , 'MR' , 1 )
# #     print(new_containers)
    
# #     # You can add logic here to compute a score for Optuna based on some objective criteria
# #     return 1  # Dummy return value

# # # Run Optuna optimization
# # study = optuna.create_study(direction='maximize')

# # study.optimize(optuna_objective, n_trials=1) 







